In [ ]:
import os
import random
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.models import resnet18, ResNet18_Weights

# --- Константы ---
SEED = 42
IMG_SIZE = 192
BATCH_SIZE = 16
EPOCHS = 3  # Меньше для примера
LR = 1e-4
WEIGHT_DECAY = 1e-5
VAL_SIZE = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# --- Трансформации ---
train_tfms = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    ToTensorV2(),
])

valid_tfms = A.Compose([
    A.LongestMaxSize(IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_CONSTANT),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

def _safe_imread(p: str) -> np.ndarray:
    img = cv2.imread(p, cv2.IMREAD_COLOR)
    if img is None:
        img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

регрессия

In [ ]:
# --- Dataset ---
class MultiModalDataset(Dataset):
    def __init__(self, df, features_cols, target_col, is_train=True):
        self.df = df.reset_index(drop=True)
        self.features_cols = features_cols
        self.target_col = target_col
        self.is_train = is_train
        self.tfms = train_tfms if is_train else valid_tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # --- Изображение ---
        img = _safe_imread(row['path_to_image'])
        img = self.tfms(image=img)["image"].contiguous()

        # --- Табличные фичи ---
        tabular_features = torch.tensor(row[self.features_cols].values.astype(np.float32), dtype=torch.float32)

        if self.is_train:
            target = torch.tensor(row[self.target_col], dtype=torch.float32)
            return img, tabular_features, target
        else:
            return img, tabular_features, row["ID"]

# --- Модель ---
class MultiModalRegressor(nn.Module):
    def __init__(self, num_tabular_features, hidden_dim=64):
        super().__init__()
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.fc_img = nn.Linear(in_features, hidden_dim)
        self.fc_tab = nn.Linear(num_tabular_features, hidden_dim)
        self.head = nn.Linear(hidden_dim * 2, 1)

    def forward(self, img, tab):
        img_emb = self.backbone(img)
        img_emb = img_emb.view(img_emb.size(0), -1)
        img_emb = torch.relu(self.fc_img(img_emb))

        tab_emb = torch.relu(self.fc_tab(tab))

        combined = torch.cat([img_emb, tab_emb], dim=1)
        out = self.head(combined)
        return out.squeeze(1)

# --- Тренировка ---
df = pd.read_csv("train.csv")
features = ["year", "mileage"]
target = "price_target"

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

train_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED)

train_ds = MultiModalDataset(train_df, features, target, is_train=True)
valid_ds = MultiModalDataset(val_df, features, target, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

model = MultiModalRegressor(num_tabular_features=len(features)).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
    for imgs, tabs, targets in pbar:
        imgs, tabs, targets = imgs.to(DEVICE), tabs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs, tabs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})

# --- Предикт и сабмит ---
model.eval()
test_df = pd.read_csv("test.csv")
test_df[features] = scaler.transform(test_df[features]) # Важно: использовать fit_transform от train
test_ds = MultiModalDataset(test_df, features, target, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

all_preds, all_ids = [], []
with torch.no_grad():
    for imgs, tabs, ids in tqdm(test_loader, desc="Predict"):
        imgs, tabs = imgs.to(DEVICE), tabs.to(DEVICE)
        preds = model(imgs, tabs)
        all_preds.append(preds.cpu())
        all_ids.extend(ids)

final_preds = torch.cat(all_preds).numpy()
submission = pd.DataFrame({"ID": all_ids, "target": final_preds})
submission.to_csv("submission_multimodal_regression.csv", index=False)
print("Submission saved for multimodal regression.")

классификация

In [ ]:
# --- Модель ---
class MultiModalClassifier(nn.Module):
    def __init__(self, num_tabular_features, num_classes, hidden_dim=64):
        super().__init__()
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.fc_img = nn.Linear(in_features, hidden_dim)
        self.fc_tab = nn.Linear(num_tabular_features, hidden_dim)
        self.head = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, img, tab):
        img_emb = self.backbone(img)
        img_emb = img_emb.view(img_emb.size(0), -1)
        img_emb = torch.relu(self.fc_img(img_emb))

        tab_emb = torch.relu(self.fc_tab(tab))

        combined = torch.cat([img_emb, tab_emb], dim=1)
        out = self.head(combined)
        return out

# --- Тренировка ---
df = pd.read_csv("train.csv")
features = ["year", "mileage"]
target = "body_type"

le = LabelEncoder()
df[target] = le.fit_transform(df[target]) # Преобразуем в числа: 0, 1, 2...
num_classes = len(le.classes_)

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

train_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED)

train_ds = MultiModalDataset(train_df, features, target, is_train=True)
valid_ds = MultiModalDataset(val_df, features, target, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

model = MultiModalClassifier(num_tabular_features=len(features), num_classes=num_classes).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
    for imgs, tabs, targets in pbar:
        imgs, tabs, targets = imgs.to(DEVICE), tabs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs, tabs)
        loss = criterion(preds, targets.long()) # Целевые метки должны быть long
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})

# --- Предикт и сабмит ---
model.eval()
test_df = pd.read_csv("test.csv")
test_df[features] = scaler.transform(test_df[features])
test_ds = MultiModalDataset(test_df, features, target, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

all_logits, all_ids = [], []
with torch.no_grad():
    for imgs, tabs, ids in tqdm(test_loader, desc="Predict"):
        imgs, tabs = imgs.to(DEVICE), tabs.to(DEVICE)
        logits = model(imgs, tabs)
        all_logits.append(torch.softmax(logits, dim=1).cpu()) # Сохраняем вероятности
        all_ids.extend(ids)

final_probs = torch.cat(all_logits).numpy()
# Пример: сохранить вероятности для каждого класса
submission = pd.DataFrame({"ID": all_ids})
for i, class_name in enumerate(le.classes_):
    submission[f"prob_{class_name}"] = final_probs[:, i]
submission.to_csv("submission_multimodal_classification.csv", index=False)
print("Submission saved for multimodal classification.")

бинарная классификация

In [ ]:
# --- Модель ---
class MultiModalBinaryClassifier(nn.Module):
    def __init__(self, num_tabular_features, hidden_dim=64):
        super().__init__()
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.fc_img = nn.Linear(in_features, hidden_dim)
        self.fc_tab = nn.Linear(num_tabular_features, hidden_dim)
        self.head = nn.Linear(hidden_dim * 2, 1)

    def forward(self, img, tab):
        img_emb = self.backbone(img)
        img_emb = img_emb.view(img_emb.size(0), -1)
        img_emb = torch.relu(self.fc_img(img_emb))

        tab_emb = torch.relu(self.fc_tab(tab))

        combined = torch.cat([img_emb, tab_emb], dim=1)
        out = self.head(combined)
        return out.squeeze(1)

# --- Тренировка ---
df = pd.read_csv("train.csv")
features = ["year", "mileage"]
target = "is_new" # 0 или 1

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

train_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED)

train_ds = MultiModalDataset(train_df, features, target, is_train=True)
valid_ds = MultiModalDataset(val_df, features, target, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

model = MultiModalBinaryClassifier(num_tabular_features=len(features)).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.BCEWithLogitsLoss() # Удобно: сигмоида + BCE в одном

model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
    for imgs, tabs, targets in pbar:
        imgs, tabs, targets = imgs.to(DEVICE), tabs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs, tabs)
        loss = criterion(preds, targets.float()) # Целевые метки должны быть float
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})

# --- Предикт и сабмит ---
model.eval()
test_df = pd.read_csv("test.csv")
test_df[features] = scaler.transform(test_df[features])
test_ds = MultiModalDataset(test_df, features, target, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

all_logits, all_ids = [], []
with torch.no_grad():
    for imgs, tabs, ids in tqdm(test_loader, desc="Predict"):
        imgs, tabs = imgs.to(DEVICE), tabs.to(DEVICE)
        logits = model(imgs, tabs)
        all_logits.append(torch.sigmoid(logits).cpu()) # Вероятность
        all_ids.extend(ids)

final_probs = torch.cat(all_logits).numpy()
submission = pd.DataFrame({"ID": all_ids, "target": final_probs.flatten()})
submission.to_csv("submission_multimodal_binary.csv", index=False)
print("Submission saved for multimodal binary classification.")

только для табличек (для яичек)

In [ ]:
# --- Dataset ---
class TabularOnlyDataset(Dataset):
    def __init__(self, df, features_cols, target_col, is_train=True):
        self.df = df.reset_index(drop=True)
        self.features_cols = features_cols
        self.target_col = target_col
        self.is_train = is_train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        tabular_features = torch.tensor(row[self.features_cols].values.astype(np.float32), dtype=torch.float32)
        if self.is_train:
            target = torch.tensor(row[self.target_col], dtype=torch.float32)
            return tabular_features, target
        else:
            return tabular_features, row["ID"]

# --- Модель ---
class TabularOnlyRegressor(nn.Module):
    def __init__(self, num_features, hidden_dim=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.network(x).squeeze(1)

# --- Тренировка ---
df = pd.read_csv("train.csv")
features = ["year", "mileage"]
target = "price_target"

scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

train_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED)

train_ds = TabularOnlyDataset(train_df, features, target, is_train=True)
valid_ds = TabularOnlyDataset(val_df, features, target, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

model = TabularOnlyRegressor(num_tabular_features=len(features)).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
    for tabs, targets in pbar:
        tabs, targets = tabs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        preds = model(tabs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})

# --- Предикт и сабмит ---
model.eval()
test_df = pd.read_csv("test.csv")
test_df[features] = scaler.transform(test_df[features])
test_ds = TabularOnlyDataset(test_df, features, target, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

all_preds, all_ids = [], []
with torch.no_grad():
    for tabs, ids in tqdm(test_loader, desc="Predict"):
        tabs = tabs.to(DEVICE)
        preds = model(tabs)
        all_preds.append(preds.cpu())
        all_ids.extend(ids)

final_preds = torch.cat(all_preds).numpy()
submission = pd.DataFrame({"ID": all_ids, "target": final_preds})
submission.to_csv("submission_tabular_only_regression.csv", index=False)
print("Submission saved for tabular-only regression.")

только изображения (для бражения)

In [ ]:
# --- Dataset ---
class ImageOnlyDataset(Dataset):
    def __init__(self, df, target_col, is_train=True):
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.is_train = is_train
        self.tfms = train_tfms if is_train else valid_tfms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = _safe_imread(row['path_to_image'])
        img = self.tfms(image=img)["image"].contiguous()
        if self.is_train:
            target = torch.tensor(row[self.target_col], dtype=torch.float32)
            return img, target
        else:
            return img, row["ID"]

# --- Модель ---
class ImageOnlyRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        backbone = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = backbone.fc.in_features
        self.backbone = nn.Sequential(*list(backbone.children())[:-1])
        self.head = nn.Linear(in_features, 1)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        return x.squeeze(1)

# --- Тренировка ---
df = pd.read_csv("train.csv")
target = "price_target"

train_df, val_df = train_test_split(df, test_size=VAL_SIZE, random_state=SEED)

train_ds = ImageOnlyDataset(train_df, target, is_train=True)
valid_ds = ImageOnlyDataset(val_df, target, is_train=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

model = ImageOnlyRegressor().to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

model.train()
for epoch in range(EPOCHS):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} Train")
    for imgs, targets in pbar:
        imgs, targets = imgs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        preds = model(imgs)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({"Loss": loss.item()})

# --- Предикт и сабмит ---
model.eval()
test_df = pd.read_csv("test.csv")
test_ds = ImageOnlyDataset(test_df, target, is_train=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

all_preds, all_ids = [], []
with torch.no_grad():
    for imgs, ids in tqdm(test_loader, desc="Predict"):
        imgs = imgs.to(DEVICE)
        preds = model(imgs)
        all_preds.append(preds.cpu())
        all_ids.extend(ids)

final_preds = torch.cat(all_preds).numpy()
submission = pd.DataFrame({"ID": all_ids, "target": final_preds})
submission.to_csv("submission_image_only_regression.csv", index=False)
print("Submission saved for image-only regression.")